<a href="https://colab.research.google.com/github/hachons/trailmet/blob/dev/experiments/pruning/chip/chip_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/hachons/trailmet.git

Cloning into 'trailmet'...
remote: Enumerating objects: 1205, done.
remote: Counting objects: 100% (571/571), done.
remote: Compressing objects: 100% (271/271), done.
remote: Total 1205 (delta 399), reused 361 (delta 295), pack-reused 634
Receiving objects: 100% (1205/1205), 2.34 MiB | 11.42 MiB/s, done.
Resolving deltas: 100% (729/729), done.


In [2]:
cd /content/trailmet

/content/trailmet


In [5]:
!git pull

remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), 5.82 KiB | 1.16 MiB/s, done.
From https://github.com/hachons/trailmet
   f45fead..2e676f5  dev        -> origin/dev
Updating f45fead..2e676f5
Fast-forward
 experiments/prune/chip.ipynb   | 356 +++++++++++++++++++++++++++++++++++++++++
 trailmet/models/resnet_chip.py | 157 ++++++++++++++++++
 2 files changed, 513 insertions(+)
 create mode 100644 experiments/prune/chip.ipynb
 create mode 100644 trailmet/models/resnet_chip.py


In [7]:
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.models import ModelsFactory
from trailmet.models.resnet_chip import resnet_50
from trailmet.datasets.classification import DatasetFactory
import yaml

In [9]:
root = "/content/trailmet/experiments/prune/chip"
from trailmet.datasets.classification import DatasetFactory
data_root = "/content/data_dir"

In [10]:
!mkdir /content/data_dir

In [11]:
train_transform = transforms.Compose(
[transforms.ToTensor()])

val_transform = transforms.Compose(
[transforms.ToTensor()])

test_transform = transforms.Compose(
[transforms.ToTensor()])

transforms1 = {
    'train': train_transform, 
    'val': val_transform, 
    'test': test_transform}
def train_target_transform(label):
    return label

def val_target_transform(label):
    return label

def test_target_transform(label):
    return label

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}


cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR100', 
                                        root = data_root,
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.2,
                                        transform = transforms1,
                                        target_transform = target_transforms
                                        )

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting /content/data_dir/cifar-100-python.tar.gz to /content/data_dir
Files already downloaded and verified
Files already downloaded and verified


In [12]:
class CFG:
    def __init__(self):
        self.batch_size = 128
        self.arch = 'resnet_50'
        self.repeat = 5
        self.ci_dir = './CI_resnet_50'
        self.lr_type = 'cos'
        self.learning_rate = 0.01
        self.epochs = 5
        self.num_layers = 53
        self.feature_map_dir = './conv_feature_map'
        self.sparsity = [0.]+[0.1]*3+[0.35]*16
        self.label_smooth = 0
        self.device = 'cuda'
        self.momentum = 0.9
        self.weight_decay = 0.005
        self.lr_decay_step = 0.9
        self.result_dir = './result'
        self.pretrain_dir = './pretrained_models/resnet_50.pt'
        self.conv_index = torch.tensor(1)

In [13]:
args = CFG()
args = args.__dict__
dataloaders = { 'train' : torch.utils.data.DataLoader(
        cifar_dataset['train'], batch_size=128, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=0
    ),
               'val':  torch.utils.data.DataLoader(
        cifar_dataset['val'], batch_size=128, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=0
    ),  
               'test':  torch.utils.data.DataLoader(
        cifar_dataset['test'], batch_size=128, 
        sampler=cifar_dataset['test_sampler'],
        num_workers=0
    )}

In [21]:
model = eval(args['arch'])(sparsity=[0.]*100).to(args['device'])
checkpoint = torch.load(args['pretrain_dir'])
model.load_state_dict(checkpoint)

FileNotFoundError: ignored

In [11]:
from trailmet.algorithms.prune.chip import Chip

In [12]:
a = Chip(model,dataloaders,**args)

In [ ]:
a.compress_model()

/content/drive/MyDrive/trailmet-dev/trailmet/algorithms/prune/chip.py:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  original_norm = torch.norm(torch.tensor(conv_reshape[i, :, :]), p='nuc').item()
